<a href="https://colab.research.google.com/github/melkatewabe10/Machine-learning_LST-Estimation-/blob/main/RF_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author: Tewabe Melkamu**

Date: 3/14/2025

RF_model

In [ ]:
%pip install rasterio

**Import necessary libraries**

In [ ]:
#Author: Tewabe Melkamu
#Date: 3/14/2025
#RF_model
"""
Script to predict Land Surface Temperature (LST) using Random Forest Regression.
This script is tailored for Google Colab. It loads GeoTIFF training data from Google Drive,
preprocesses the data, trains the model (ntree=200, mtry=4),
evaluates model performance (OOB, RMSE, MAE, R2), and predicts the entire image.
Additional code sections provide data visualizations and save evaluation metrics.
"""

# =============================================================================
# 1. Import Necessary Libraries
# =============================================================================
import os
import time
import pandas as pd
import numpy as np
from scipy import stats
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

# Additional libraries for visualization and file operations
from collections import OrderedDict


**Define Data Folder and File Paths**

In [ ]:
# =============================================================================
# Define Data Folder and File Paths
# =============================================================================
# Update the folder path below to the directory where your .tif files reside on your Drive.
data_folder = '/content/drive/MyDrive/Taiwan_Yearly /Taiwan_2024'

# Create a dictionary to store file paths for each predictor variable.
predictor_files = OrderedDict({
    'NDVI':   os.path.join(data_folder, 'NDVI_2024.tif'),
    'EVI': os.path.join(data_folder, 'EVI_2024.tif'),
    'NDWI':  os.path.join(data_folder, 'NDWI_2024.tif'),
    'LAI':   os.path.join(data_folder, 'LAI_2024.tif'),
    'ALB':  os.path.join(data_folder, 'ALB_2024.tif'),
    'ELV':   os.path.join(data_folder, 'ELV.tif'),
    'SLP':   os.path.join(data_folder, 'SLP.tif'),
    'DSR':   os.path.join(data_folder, 'DSR_2024.tif')
})

# Define file path for target variable (LST)
lst_path = os.path.join(data_folder, 'LST_2024.tif')

# Check that all files exist and print a warning if any are missing.
print("Verifying file paths...")
for key, path in predictor_files.items():
    if not os.path.exists(path):
        print(f"Warning: {key} file not found at {path}")
    else:
        print(f"{key} file found.")
if not os.path.exists(lst_path):
    print(f"Warning: LST file not found at {lst_path}")
else:
    print("LST file found.")
print("\n")

**Define Functions for Raster Operations and Visualizations**

In [ ]:
# =============================================================================
#  Define Functions for Raster Operations and Visualizations
# =============================================================================
def read_raster(raster_path):
    """
    Reads a single-band raster file and returns its data and metadata.

    Parameters:
        raster_path (str): Path to the raster file.

    Returns:
        data (np.ndarray): 2D array of raster data.
        meta (dict): Raster metadata (projection, transform, etc.).
    """
    with rasterio.open(raster_path) as src:
        data = src.read(1)  # Assuming a single-band raster.
        meta = src.meta
    return data, meta

def plot_histogram(data, title, bins=50):
    """
    Plots a histogram of the input data.

    Parameters:
        data (np.ndarray): 1D array of data values.
        title (str): Title for the histogram.
        bins (int): Number of bins in the histogram.
    """
    plt.figure(figsize=(8, 5))
    plt.hist(data[np.isfinite(data)], bins=bins, color='gray', edgecolor='black')
    plt.title(title)
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

def plot_scatter(x, y, xlabel, ylabel, title):
    """
    Plots a scatter plot of two variables.

    Parameters:
        x (np.ndarray): x-axis values.
        y (np.ndarray): y-axis values.
        xlabel (str): Label for x-axis.
        ylabel (str): Label for y-axis.
        title (str): Plot title.
    """
    plt.figure(figsize=(8, 5))
    plt.scatter(x, y, alpha=0.5, s=10)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)
    plt.show()

# =============================================================================
#  Example: Read a Raster and Display a Plot
# =============================================================================
# Read the NDVI raster as an example
ndvi_path = predictor_files['DSR']
ndvi_data, ndvi_meta = read_raster(ndvi_path)

# Plot the histogram for NDVI data
#plot_histogram(ndvi_data, "Histogram of NDVI_2020")

**Read All Predictor and Target Rasters**

In [ ]:
# =============================================================================
# Read All Predictor and Target Rasters
# =============================================================================
print("Reading predictor rasters...")
predictors = {}
meta = None
for key, path in predictor_files.items():
    data, meta = read_raster(path)
    predictors[key] = data
    print(f"Read {key} with shape: {data.shape}")

print("\nReading target raster (LST)...")
lst, _ = read_raster(lst_path)
print("LST shape:", lst.shape)
print("\n")


In [ ]:
# =============================================================================
#  Check Consistency of Raster Shapes
# =============================================================================
shapes = [data.shape for data in predictors.values()] + [lst.shape]
if len(set(shapes)) != 1:
    print("Error: Raster files do not have the same dimensions. Please check your data.")
else:
    print("All raster files have consistent dimensions.\n")

All raster files have consistent dimensions.



**Flatten Rasters and Stack Features**

In [ ]:
# =============================================================================
# Flatten Rasters and Stack Features
# =============================================================================
print("Preparing data for modeling...")
n_pixels = lst.size  # Total number of pixels (assumes all rasters share the same shape)

# Flatten each predictor and store in a list following the order in predictor_files.
feature_list = []
for key in predictor_files.keys():
    flat_array = predictors[key].flatten()
    feature_list.append(flat_array)
    # Plot histogram for each predictor (optional, for initial data inspection)
    #plot_histogram(flat_array, f"Histogram of {key}")

# Convert list of arrays to 2D feature array (rows: pixels, columns: features)
features = np.vstack(feature_list).T

# Flatten target variable
lst_flat = lst.flatten()

# =============================================================================
# Create a Valid Data Mask (Filter Out Invalid Pixels)
# =============================================================================
# Here, valid data are those with finite values for all predictors and the target.
valid_mask = np.isfinite(features).all(axis=1) & np.isfinite(lst_flat)
print("Total pixels:", n_pixels)
print("Valid pixels for training:", np.sum(valid_mask))
print("\n")

# =============================================================================
# Filter Features and Target Data
# =============================================================================
features_valid = features[valid_mask]
lst_valid      = lst_flat[valid_mask]

# =============================================================================
# Optional: Inspect Data Distribution for the Target
# =============================================================================
#plot_histogram(lst_valid, "Histogram of LST (Valid Pixels)")


**Model** **Training**

In [ ]:
# =============================================================================
# Set Up Random Forest Model Parameters (Using Tuned Hyperparameters)
# =============================================================================
n_estimators = 200
max_features = 4
min_samples_split = 2
max_depth = None
random_state = 42

# Initialize and Train the Random Forest Regressor with Tuned Parameters
# =============================================================================
print("Initializing the Random Forest regressor with tuned hyperparameters...")
rf_model = RandomForestRegressor(n_estimators=n_estimators,
                                 max_features=max_features,
                                 min_samples_split=min_samples_split,
                                 max_depth=max_depth,
                                 oob_score=True,
                                 random_state=random_state,
                                 n_jobs=-1)

print("Training the model...")
start_time = time.time()
rf_model.fit(features_valid, lst_valid)
elapsed_time = time.time() - start_time
print(f"Training completed in {elapsed_time:.2f} seconds.\n")


**Evaluate** **the** **Model**

In [ ]:
# =============================================================================
# Evaluate the Model Using Out-of-Bag Predictions
# =============================================================================
oob_score = rf_model.oob_score_
print("OOB Score:", oob_score)

# OOB predictions for evaluation
lst_oob_pred = rf_model.oob_prediction_
rmse = np.sqrt(mean_squared_error(lst_valid, lst_oob_pred))
mae = mean_absolute_error(lst_valid, lst_oob_pred)
r2 = r2_score(lst_valid, lst_oob_pred)

print("\nEvaluation Metrics:")
print("RMSE:", rmse)
print("MAE:", mae)
print("R2 Score:", r2)
print("\n")

# =============================================================================
# Save Evaluation Metrics to a CSV File
# =============================================================================
import pandas as pd

# Create a dictionary with evaluation metrics
evaluation_metrics = {
    'OOB_Score': [oob_score],
    'RMSE': [rmse],
    'MAE': [mae],
    'R2': [r2]
}

# Convert the dictionary to a DataFrame
df_eval = pd.DataFrame(evaluation_metrics)

# Define the CSV output path (adjust data_folder as needed)
csv_path = os.path.join(data_folder, 'evaluation_metrics.csv')

# Save the DataFrame to CSV
df_eval.to_csv(csv_path, index=False)
print("Evaluation metrics saved to:", csv_path)


**Scatter Plot: Observed vs.Predicted LST**

In [ ]:
# =============================================================================
# Scatter Plot: Observed vs. OOB Predicted LST (for training pixels)
# =============================================================================
plot_scatter(lst_valid, lst_oob_pred, "Observed LST", "OOB Predicted LST",
             "Observed vs. OOB Predicted LST (Clipped)")

**Compute Feature Importances**

In [ ]:
# =============================================================================
# Compute and Plot Feature Importances
# =============================================================================
importances = rf_model.feature_importances_
feature_names = list(predictor_files.keys())

plt.figure(figsize=(10, 6))
bars = plt.bar(range(len(importances)), importances, tick_label=feature_names)
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.title("Random Forest Feature Importances")
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show()

# =============================================================================
# Save Feature Importances to a CSV File
# =============================================================================
# Create a DataFrame with feature names and their corresponding importance values.
df_features = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Define the CSV output path (adjust data_folder if necessary)
csv_feature_importance = os.path.join(data_folder, 'feature_importances.csv')

# Save the DataFrame to a CSV file without the index.
df_features.to_csv(csv_feature_importance, index=False)
print("Feature importances saved to:", csv_feature_importance)


**Predict LST for the Entire Image**

In [ ]:
# =============================================================================
# Predict LST for the Entire Image
# =============================================================================
print("Predicting LST for the entire image...")
predicted_lst = rf_model.predict(features)
predicted_lst_image = predicted_lst.reshape(lst.shape)

# =============================================================================
#Handle Invalid Pixels in the Predicted Output
# =============================================================================
# For pixels that were originally invalid (e.g., nodata), assign a nodata value.
predicted_lst_image[~np.isfinite(predictors['ELV'])] = np.nan

# =============================================================================
# Save the Predicted LST as a New GeoTIFF File
# =============================================================================
output_path = os.path.join(data_folder, 'predicted_LST.tif')
meta.update(dtype=rasterio.float32, count=1)

with rasterio.open(output_path, 'w', **meta) as dst:
    dst.write(predicted_lst_image.astype(rasterio.float32), 1)
print("Predicted LST image saved to:", output_path)
print("\n")

**Compute the Difference_ "e" value**

In [ ]:
# =============================================================================
# New Step: Compute and Save the Difference Image
# =============================================================================
# Compute the difference between the original LST and the predicted LST
difference_image = lst - predicted_lst_image

# Save the difference image as a new GeoTIFF file
difference_output_path = os.path.join(data_folder, 'LST_e.tif')
meta.update(dtype=rasterio.float32, count=1)

with rasterio.open(difference_output_path, 'w', **meta) as dst:
    dst.write(difference_image.astype(rasterio.float32), 1)
print("Difference image (Original LST - Predicted LST) saved to:", difference_output_path)
print("\n")



**Visualization**

In [ ]:
# =============================================================================
# Visualize the Original, Predicted, and Difference Images Side by Side
# =============================================================================
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8))

# Original LST image (clipped)
cax1 = ax1.imshow(lst, cmap='viridis')
ax1.set_title("Original LST")
ax1.axis('off')
fig.colorbar(cax1, ax=ax1, fraction=0.046, pad=0.04)

# Predicted LST image (clipped)
cax2 = ax2.imshow(predicted_lst_image, cmap='viridis')
ax2.set_title("Predicted LST")
ax2.axis('off')
fig.colorbar(cax2, ax=ax2, fraction=0.046, pad=0.04)

# Difference image (Original - Predicted)
# Set the color limits symmetric around zero for better visualization of positive/negative errors.
diff_abs_max = np.nanmax(np.abs(difference_image))
cax3 = ax3.imshow(difference_image, cmap='RdBu', vmin=-diff_abs_max, vmax=diff_abs_max)
ax3.set_title("Difference: e image")
ax3.axis('off')
fig.colorbar(cax3, ax=ax3, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


**Predict LST Using New Data**

In [ ]:
# =============================================================================
# NEW: Predict LST Using New Data
# =============================================================================
print("Predicting LST using new data...")

# Define the new data folder and file paths for the new predictor data.
new_data_folder = '/content/drive/MyDrive/your_new_data_folder'

new_predictor_files = OrderedDict({
    'elv':   os.path.join(new_data_folder, 'elv_new.tif'),
    'slope': os.path.join(new_data_folder, 'slope_new.tif'),
    'NDVI':  os.path.join(new_data_folder, 'NDVI_new.tif'),
    'EVI':   os.path.join(new_data_folder, 'EVI_new.tif'),
    'NDWI':  os.path.join(new_data_folder, 'NDWI_new.tif'),
    'LAI':   os.path.join(new_data_folder, 'LAI_new.tif'),
    'ALB':   os.path.join(new_data_folder, 'ALB_new.tif'),
    'DSR':   os.path.join(new_data_folder, 'DSR_new.tif')
})

# =============================================================================
# Read New Predictor Rasters
# =============================================================================
new_predictors = {}
new_meta = None
for key, path in new_predictor_files.items():
    data, new_meta = read_raster(path)
    new_predictors[key] = data
    print(f"Read new {key} with shape: {data.shape}")

# =============================================================================
# Check Consistency of New Raster Shapes
# =============================================================================
new_shapes = [data.shape for data in new_predictors.values()]
if len(set(new_shapes)) != 1:
    print("Error: New raster files do not have the same dimensions. Please check your data.")
else:
    print("All new raster files have consistent dimensions.\n")



**Predict LST for the New Data Using the Trained Model**

In [ ]:
# =============================================================================
# Flatten New Rasters and Stack Features
# =============================================================================
new_feature_list = []
for key in new_predictor_files.keys():
    flat_array = new_predictors[key].flatten()
    new_feature_list.append(flat_array)
# Create a 2D array where each row represents a pixel and each column a predictor.
new_features = np.vstack(new_feature_list).T

# =============================================================================
# Predict LST for the New Data Using the Trained Model
# =============================================================================
new_predicted_lst = rf_model.predict(new_features)
# Reshape the 1D predictions back into the 2D image shape (assumed identical for all predictors)
new_predicted_lst_image = new_predicted_lst.reshape(new_predictors[list(new_predictor_files.keys())[0]].shape)

# =============================================================================
# Handle Invalid Pixels in the New Prediction
# =============================================================================
# For example, if the new elevation data has invalid pixels, mark these as NaN in the prediction.
new_predicted_lst_image[~np.isfinite(new_predictors['elv'])] = np.nan

# =============================================================================
# Save the New Predicted LST as a GeoTIFF File
# =============================================================================
new_output_path = os.path.join(new_data_folder, 'predicted_LST_new.tif')
new_meta.update(dtype=rasterio.float32, count=1)

with rasterio.open(new_output_path, 'w', **new_meta) as dst:
    dst.write(new_predicted_lst_image.astype(rasterio.float32), 1)
print("New predicted LST image saved to:", new_output_path)




**Visualize the New Predicted LST Image**

In [ ]:
# =============================================================================
# Visualize the New Predicted LST Image
# =============================================================================
plt.figure(figsize=(10, 10))
plt.imshow(new_predicted_lst_image, cmap='viridis')
plt.title("Predicted LST (New Data)")
plt.axis('off')
plt.colorbar(label='LST Value')
plt.show()

**Final Normalized LST**

In [ ]:
# "new_predicted_lst_image" is the prediction from new data.
adjusted_new_pred = new_predicted_lst_image + difference_image

# =============================================================================
# Save the Adjusted New Prediction as a New GeoTIFF File
# =============================================================================
adjusted_output_path = os.path.join(new_data_folder, 'Normalized LST.tif')
new_meta.update(dtype=rasterio.float32, count=1)
with rasterio.open(adjusted_output_path, 'w', **new_meta) as dst:
    dst.write(adjusted_new_pred.astype(rasterio.float32), 1)
print("Normalized LST image saved to:", adjusted_output_path)

# =============================================================================
# Plot the Adjusted New Prediction
# =============================================================================
plt.figure(figsize=(10, 10))
plt.imshow(adjusted_new_pred, cmap='viridis')
plt.title("Adjusted New Predicted LST")
plt.axis('off')
plt.colorbar(label='LST Value')
plt.show()



**Compute Evaluation Metrics for final normalization**

In [ ]:
# =============================================================================
# Evaluate Adjusted New Prediction Accuracy Across the Entire Image
# =============================================================================
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

# Flatten both arrays for metric calculation
true_lst_flat = lst.flatten()
adjusted_pred_flat = adjusted_new_pred.flatten()

# Mask out invalid pixels (where ground truth or adjusted prediction is invalid)
valid_mask = np.isfinite(true_lst_flat) & np.isfinite(adjusted_pred_flat)

# Apply mask
y_true = true_lst_flat[valid_mask]
y_pred = adjusted_pred_flat[valid_mask]

# Calculate evaluation metrics
r2_adj = r2_score(y_true, y_pred)
rmse_adj = np.sqrt(mean_squared_error(y_true, y_pred))
mae_adj = mean_absolute_error(y_true, y_pred)
pearson_corr, p_value = pearsonr(y_true, y_pred)

# Print results
print("Adjusted New Prediction Evaluation Metrics (Valid Pixels Only):")
print("Pearson Correlation:", pearson_corr)
print("p-value:", p_value)
print("R² Score:", r2_adj)
print("RMSE:", rmse_adj)
print("MAE:", mae_adj)

# =============================================================================
# 20. Save Adjusted Prediction Metrics to CSV
# =============================================================================

# Define output file path
csv_output_path = os.path.join(new_data_folder, 'adjusted_prediction_metrics.csv')

# Create a DataFrame with the metrics
metrics_df = pd.DataFrame({
    'Pearson_Correlation': [pearson_corr],
    'p_value': [p_value],
    'R2_Score': [r2_adj],
    'RMSE': [rmse_adj],
    'MAE': [mae_adj]
})

# Save the DataFrame to CSV
metrics_df.to_csv(csv_output_path, index=False)

print("Adjusted prediction evaluation metrics saved to:", csv_output_path)


**correlation Matrix Example **

In [ ]:
import pandas as pd

# =============================================================================
#  Evaluate Prediction Accuracy Across the Entire Image
# =============================================================================

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Flatten both arrays for metric calculation
true_lst_flat = lst.flatten()
predicted_lst_flat = predicted_lst_image.flatten()

# Mask out invalid pixels (where elevation or LST was invalid)
valid_mask = np.isfinite(true_lst_flat) & np.isfinite(predicted_lst_flat)

# Apply mask
y_true = true_lst_flat[valid_mask]
y_pred = predicted_lst_flat[valid_mask]

# Calculate metrics
r2 = r2_score(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

# Print results
print("Full Image Evaluation Metrics (Valid Pixels Only):")
print("R² Score:", r2)
print("RMSE:", rmse)
print("MAE:", mae)

# =============================================================================
# Save Metrics to CSV
# =============================================================================

# Define output file path
csv_output_path = os.path.join(data_folder, 'LST_evaluation_metrics.csv')

# Create a DataFrame with the metrics
metrics_df = pd.DataFrame({
    'R2_Score': [r2],
    'RMSE': [rmse],
    'MAE': [mae]
})

# Save to CSV
metrics_df.to_csv(csv_output_path, index=False)

print("Evaluation metrics saved to:", csv_output_path)


Full Image Evaluation Metrics (Valid Pixels Only):
R² Score: 0.9900307302644273
RMSE: 0.4372856628712171
MAE: 0.26704630066368046
Evaluation metrics saved to: /content/drive/MyDrive/Taiwan_Yearly /Taiwan_2024/LST_evaluation_metrics.csv


# Correlation plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_scatter(true_values, predicted_values, xlabel, ylabel, title, doy=None):
    """
    Create a scatter density plot of observed vs. predicted LST with fixed axis limits.

    Parameters:
    - true_values: Array of original LST values
    - predicted_values: Array of predicted LST values (e.g., OOB or full image)
    - xlabel: Label for the X-axis
    - ylabel: Label for the Y-axis
    - title: Plot title
    - doy: Day of Year (optional, for labeling)
    """

    plt.figure(figsize=(6, 5))

    # Create a density scatter plot using hexbin
    hb = plt.hexbin(true_values, predicted_values, gridsize=70, cmap="jet", bins="log", mincnt=1)

    # Colorbar to show density levels
    cb = plt.colorbar(hb)
    cb.set_label('Count')

    # Plot 1:1 reference line from (270, 270) to (310, 310)
    plt.plot([275, 310], [275, 310], 'r-', linewidth=1)

    # Set axis limits
    plt.xlim(275, 310)
    plt.ylim(275, 310)

    # Compute performance metrics
    r2 = np.corrcoef(true_values, predicted_values)[0, 1] ** 2
    rmse = np.sqrt(np.mean((true_values - predicted_values) ** 2))

    # Add text for R² and RMSE in the upper-left corner
    plt.text(285, 302, f'R²={r2:.2f}\nRMSE={rmse:.2f}K', fontsize=12, color='black')

    # Labels and Title
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if doy:
        plt.title(f"{title}\nDOY {doy}")
    else:
        plt.title(title)

    plt.show()
import matplotlib.pyplot as plt
import numpy as np

def plot_scatter(true_values, predicted_values, xlabel, ylabel, title, doy=None):
    """
    Create a scatter density plot of observed vs. predicted LST with fixed axis limits.

    Parameters:
    - true_values: Array of original LST values
    - predicted_values: Array of predicted LST values (e.g., OOB or full image)
    - xlabel: Label for the X-axis
    - ylabel: Label for the Y-axis
    - title: Plot title
    - doy: Day of Year (optional, for labeling)
    """

    plt.figure(figsize=(6, 5))

    # Create a density scatter plot using hexbin
    hb = plt.hexbin(true_values, predicted_values, gridsize=70, cmap="jet", bins="log", mincnt=1)

    # Colorbar to show density levels
    cb = plt.colorbar(hb)
    cb.set_label('Count')

    # Plot 1:1 reference line from (270, 270) to (310, 310)
    plt.plot([280, 310], [280, 310], 'r-', linewidth=1)

    # Set axis limits
    plt.xlim(280, 310)
    plt.ylim(280, 310)

    # Compute performance metrics
    r2 = np.corrcoef(true_values, predicted_values)[0, 1] ** 2
    rmse = np.sqrt(np.mean((true_values - predicted_values) ** 2))

    # Add text for R² and RMSE in the upper-left corner
    plt.text(282, 306, f'R²={r2:.2f}\nRMSE={rmse:.2f}K', fontsize=12, color='black')

    # Labels and Title
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    #if doy:
     #   plt.title(f"{title}\nDOY {doy}")
   # else:
    #    plt.title(title)

    plt.show()

# Example Usage:
plot_scatter(lst_valid, lst_oob_pred, "Original LST (K)", "Estimated LST (K)","Correlation", doy=48)

**Considerations**

In [ ]:

# =============================================================================
#  Final Considerations and Next Steps
# =============================================================================
# In this script, we critically examined the following:
# - Data consistency: ensuring all GeoTIFFs have the same dimensions.
# - Data filtering: removing pixels with non-finite values.
# - Model evaluation: using OOB score, RMSE, MAE, and R2.
# - Visualization: comparing original vs. predicted LST and analyzing feature importances.
#
# Further steps you may consider:
# - Cross-validation with a separate test set.
# - Advanced hyperparameter tuning (e.g., grid search).
# - More robust handling of nodata values and spatial outliers.
# - Exploring additional predictors or ensemble methods.


In [ ]:
# =============================================================================
# Cross-validation with a Separate Test Set
# =============================================================================
from sklearn.model_selection import train_test_split

# Split valid data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features_valid, lst_valid,
                                                    test_size=0.2, random_state=42)

print("Data split into training and test sets.")
print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])
print("\n")

# Train a Random Forest model on the training set (without using OOB here)
rf_cv_model = RandomForestRegressor(n_estimators=n_estimators,
                                    max_features=max_features,
                                    random_state=random_state,
                                    n_jobs=-1)
print("Training the model on the training set...")
rf_cv_model.fit(X_train, y_train)

# Evaluate the model on the test set
y_test_pred = rf_cv_model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print("Test Set Evaluation Metrics:")
print("RMSE:", rmse_test)
print("MAE:", mae_test)
print("R2 Score:", r2_test)
print("\n")



Data split into training and test sets.
Training set size: 26876
Test set size: 6720


Training the model on the training set...
Test Set Evaluation Metrics:
RMSE: 0.7823354287657159
MAE: 0.5965739514146434
R2 Score: 0.9671603613701477


